# Yandex.Music

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
En este proyecto, se compararon las preferencias musicales de las ciudades de Springfield y Shelbyville. Se estudiaron datos reales de Yandex.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo: 
Prueba de tres hipótesis: 
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad. 
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche. 
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas 
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos así que necesitarás examinarlos antes de probar las hipótesis. 

Primero, se evaluó la calidad de los datos y verás si los problemas son significativos. Durante el preprocesamiento de datos, se tomó en cuenta los problemas más críticos.
 
El proyecto consistió en tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis
 
[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Abrimos los datos en Yandex.Music y examinamos.

In [42]:
import pandas as pd # importando pandas

Leemos el archivo `music_project_en.csv` de la carpeta `/datasets/` y guardamos en la variable `df`:

In [43]:
df = pd.read_csv('/datasets/music_project_en.csv')# leyendo el archivo y almacenándolo en df
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


Imprimimos las 10 primeras filas de la tabla:

In [44]:
df.head(10) # obteniendo las 10 primeras filas de la tabla df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [45]:
df.info() # obteniendo información general sobre los datos en df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. Hay un nombre con dos palabras.

El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.


### Conclusiones <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista. 

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

[Volver a Contenidos](#back)

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>
Corregimos el formato en los encabezados de las columnas y nos ocupamos de los valores ausentes. Después, comprobamos si hay duplicados en los datos.

### Estilo del encabezado <a id='header_style'></a>
Imprimimos el encabezado de la columna:

In [46]:
df.columns# la lista de los nombres de las columnas en la tabla df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Cambiamos los nombres de las columnas de acuerdo con las reglas del buen estilo:
* Si el nombre tiene varias palabras, utiliza snake_case
* Todos los caracteres deben ser minúsculas
* Elimina los espacios

In [47]:
df= df.rename(
    columns={
        '  userID':'user_id',
        'Track':'track',
        '  City  ':'city',
        'Day':'day',
    }
)# renombra las columnas

In [48]:
df.columns# comprobando el resultado: la lista de los nombres de las columnas

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Volver a Contenidos](#back)

### Valores ausentes <a id='missing_values'></a>


In [49]:
print(df.isna().sum())# calculando valores ausentes

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


Reemplazamos los valores ausentes en `'track'`, `'artist'`, y `'genre'` con la string `'unknown'`. Para ello, creamos la lista `columns_to_replace`, la recorremos con un bucle `for` y reemplazamos los valores ausentes en cada una de las columnas:

In [50]:
columns_to_replace=['track','artist','genre']
for element in columns_to_replace:
    df[element]=df[element].fillna('unknown')

# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'

In [51]:
df.isna().sum()# contando valores ausentes

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Volver a Contenidos](#back)

### Duplicados <a id='duplicates'></a>
Encontramos el número de duplicados obvios en la tabla utilizando un comando:

In [52]:
df.duplicated().sum()# contando duplicado obvios

3826

Nos deshacemos de los duplicados obvios:

In [53]:
df = df.drop_duplicates().reset_index(drop=True) # eliminando duplicados obvios

Cuenta los duplicados obvios una vez más para asegurarte de que todos han sido eliminados:

In [54]:
df.duplicated().sum()# comprobando duplicados

0

Ahora nos deshacemos de los duplicados implícitos en la columna genre. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar a resultado.

Imprimimos una lista de nombres únicos de géneros, ordenados en orden alfabético.

In [55]:
sorted(df['genre'].unique())# inspeccionando los nombres de géneros únicos

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

Buscamos en la lista para encontrar duplicados implícitos del género `hiphop`. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

Para deshacerte de ellos, declaramos la función `replace_wrong_genres()` con dos parámetros: 
* `wrong_genres=` — la lista de duplicados
* `correct_genre=` — la string con el valor correcto

La función debería corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, remplaza cada valor de la lista `wrong_genres` con el valor en `correct_genre`.

In [56]:
def replace_wrong_genres(wrong_genres,correct_genre): # función para reemplazar duplicados implícitos
    for element in wrong_genres:
        df['genre']=df['genre'].replace(element,correct_genre)

In [57]:
duplicates=['hip','hop','hip-hop']
replace_wrong_genres(duplicates,'hiphop')# eliminando duplicados implícitos

In [58]:
sorted(df['genre'].unique())# revisando en busca de duplicados implícitos

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

[Volver a Contenidos](#back)

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por `'unknown'`. Pero todavía tenemos que ver si los valores ausentes en `'genre'` afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

[Volver a Contenidos](#back)

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Comprobamos esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

* Dividimos a los usuarios en grupos por ciudad.
* Comparamos cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.


Evaluamos la actividad del usuario en cada ciudad. Agrupamos los datos por ciudad y encontramos el número de canciones reproducidas en cada grupo.



In [59]:
print(df.groupby('city')['track'].count())# contando las pistas reproducidas en cada ciudad

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64


Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.

Agrupamos los datos por día de la semana y encontramos el número de pistas reproducidas el lunes, miércoles y viernes.


In [60]:
print(df.groupby('day')['track'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.

Creamos la función `number_tracks()` para calcular el número de canciones reproducidas en un determinado día y ciudad. Requerirá dos parámetros:
* día de la semana
* nombre de la ciudad

En la función, utiliza una variable para almacenar las filas de la tabla original, donde:
  * el valor de la columna `'day'` es igual al parámetro de día
  * el valor de la columna `'city'` es igual al parámetro de ciudad

Aplicamos un filtrado consecutivo con indexación lógica.

Después, calculamos los valores de la columna `'user_id'` en la tabla resultante. Almacenamos el resultado en la nueva variable. Recuperamos esta variable de la función.

In [61]:
def number_tracks(day,city): 
    track_list=df[df['city']==city]
    track_list=track_list[track_list['day']==day] 
    track_list_count=track_list['user_id'].count()
    return track_list_count                 

Llama a `number_tracks()` seis veces, cambiando los valores de los parámetros, para que recuperes los datos de ambas ciudades para cada uno de los tres días.

In [62]:
print(number_tracks('Monday','Springfield'))# el número de canciones reproducidas en Springfield el lunes

15740


In [63]:
print(number_tracks('Monday','Shelbyville'))# el número de canciones reproducidas en Shelbyville el lunes

5614


In [64]:
print(number_tracks('Wednesday','Springfield'))# el número de canciones reproducidas en Springfield el miércoles

11056


In [65]:
print(number_tracks('Wednesday','Shelbyville'))# el número de canciones reproducidas en Shelbyville el miércoles

7003


In [66]:
print(number_tracks('Friday','Springfield'))# el número de canciones reproducidas en Springfield el viernes

15945


In [67]:
print(number_tracks('Friday','Shelbyville'))# el número de canciones reproducidas en Shelbyville el viernes

5895


Utilizamos `pd.DataFrame` para crear una tabla, donde
* Los nombres de las columnas son: `['city', 'monday', 'wednesday', 'friday']`
* Los datos son los resultados que conseguiste de `number_tracks()`

In [68]:
tabla=[('Springfield',number_tracks('Monday','Springfield'),number_tracks('Wednesday','Springfield'),number_tracks('Friday','Springfield')),('Shelbyville',number_tracks('Monday','Shelbyville'),number_tracks('Wednesday','Shelbyville'),number_tracks('Friday','Shelbyville'))]
tabla=pd.DataFrame(data=tabla, columns=['city', 'monday', 'wednesday', 'friday'])

In [69]:
tabla 

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

[Volver a Contenidos](#back)

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Obtenemos tablas:
* Para Springfield — `spr_general`
* Para Shelbyville — `shel_general`

In [70]:
spr_general=df[df['city']=='Springfield']# obteniendo la tabla spr_general de las filas de df, 
# donde los valores en la columna 'city' es 'Springfield'

In [71]:
shel_general=df[df['city']=='Shelbyville']# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'

Escribimos la función genre_weekday() con cuatro parámetros:
* Una tabla para los datos (`df`)
* El día de la semana (`day`)
* La marca de fecha y hora en formato 'hh:mm' (`time1`)
* La marca de fecha y hora en formato 'hh:mm' (`time2`)

La función debería devolver información de los 15 géneros más populares de un día determinado en un período entre dos marcas de fecha y hora.

In [72]:
def genre_weekday(df,day,time1,time2:str):
    
  # genre_df solo almacenará aquellas filas df en las que el día sea igual a day=
    genre_df =df[(df['day']==day)] # escribe tu código aquí

    # genre_df solo almacenará aquellas filas df en las que el tiempo sea menos que time2=
    genre_df = genre_df[pd.to_datetime(genre_df['time']).dt.hour>=time1]# escribe tu código aquí

    # genre_df solo almacenará aquellas filas df en las que el tiempo sea mayor que time1=
    genre_df = genre_df[pd.to_datetime(genre_df['time']).dt.hour<time2]# escribe tu código aquí

    # agrupa el DataFrame filtrado por la columna con los nombres de los géneros, toma la columna de género, y encuentra el número de filas por cada género con el método count()
    genre_df_grouped = genre_df.groupby('genre').count()

    # ordenaremos el resultado en orden descendente (por lo que los géneros más populares aparecerán primero en el objeto Series)
    genre_df_sorted = genre_df_grouped['user_id'].sort_values(ascending=False)

    # devolveremos el objeto Series que almacena los 15 géneros más populares en un día determinado en un periodo de tiempo determinado
    return genre_df_sorted[:15]

Compara los resultados de la función `genre_weekday()`para Springfield y Shelbyville el lunes por la mañana (de 7 a 11) y el viernes por la tarde (de 17:00 a 23:00):

In [73]:
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)
genre_weekday(spr_general,'Monday',7,11)

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: user_id, dtype: int64

In [74]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)
genre_weekday(shel_general,'Monday',7,11)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: user_id, dtype: int64

In [75]:
# llamando a la función para el viernes por la tarde en Springfield
print(genre_weekday(spr_general,'Friday',17,23))

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: user_id, dtype: int64


In [76]:
# llamando a la función para el viernes por la tarde en Shelbyville
print(genre_weekday(shel_general,'Friday',17,23))

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: user_id, dtype: int64


**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

[Volver a Contenidos](#back)

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

Agrupamos la tabla `spr_general` por género y encontramos el número de canciones reproducidas de cada género con el método `count()`. Después, ordenamos el resultado en orden descendente y guardamos en `spr_genres`.

In [77]:
spr_genres=spr_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Imprime las 10 primeras filas de `spr_genres`:

In [78]:
print(spr_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Ahora hacemos lo mismo con los datos de Shelbyville.

Agrupamos la tabla `shel_general` por género y encontramos el número de canciones reproducidas de cada género. Después, ordenamos el resultado en orden descendente y guardamos en la tabla `shel_genres`:


In [79]:
shel_genres=shel_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Imprime las 10 primeras filas de `shel_genres`:

In [80]:
# imprimiendo las 10 primeras filas de shel_genres
print(shel_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades. 
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas. 

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.


[Volver a Contenidos](#back)